In [1]:
import cv2
import numpy as np
print(cv2.__version__)

4.7.0


## Загрузка изображения

In [2]:
train = np.loadtxt('data/digit/train.csv', delimiter=',', skiprows=1)
# сохраняем разметку в отдельную переменную
train_label = train[:, 0]
# приводим размерность к удобному для обаботки виду
train_img = np.resize(train[:, 1:], (train.shape[0], 28, 28))

In [3]:
test = np.loadtxt('data/digit/test.csv', delimiter=',', skiprows=1)
# приводим размерность к удобному для обаботки виду
test_img = np.resize(test, (test.shape[0], 28, 28))

## Задаем параметры для вычисления дескриптора

In [4]:
winSize = (28,28)
blockSize = (14,14)
blockStride = (14,14)
cellSize = (7,7)
nbins = 18
derivAperture = 1
winSigma = 4.
histogramNormType = 0
L2HysThreshold = 2.0000000000000001e-01
gammaCorrection = 0
nlevels = 64

hog = cv2.HOGDescriptor(winSize,
                        blockSize,
                        blockStride,
                        cellSize,
                        nbins,
                        derivAperture,
                        winSigma,
                        histogramNormType,
                        L2HysThreshold,
                        gammaCorrection,
                        nlevels)

## Вычисляем дескриптор для изображений

In [5]:
# тренировочные данные
train_descriptor = []
for img in train_img:
    descriptor = hog.compute(np.uint8(img))
    train_descriptor.append(descriptor)

In [6]:
# тестовые данные
test_descriptor = []
for img in test_img:
    descriptor = hog.compute(np.uint8(img))
    test_descriptor.append(descriptor)

## Создадим и обучим классификатор

In [7]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_jobs=16, n_estimators=500, max_depth=10)

classifier.fit(train_descriptor, train_label)

RandomForestClassifier(max_depth=10, n_estimators=500, n_jobs=16)

## Сформируем предсказанные классы и оценим точность классификатора

In [8]:
pred = classifier.predict(test_descriptor)

In [9]:
import pandas as pd

submission=pd.DataFrame()

imgid=[]
for i in range(1, len(pred)+1):
    imgid.append(i)

submission["Imageid"]=imgid
submission["Label"]=pred
submission["Label"] = submission["Label"].apply(int)
submission.head()

,Imageid,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3


In [10]:
submission.to_csv("submission.csv", index=False)

Раздел, для проверки своих результатов на тестовых данных и лучших результатов с Kaggle

In [11]:
test_labels = np.loadtxt('submission_best.csv', delimiter=',', skiprows=1)

In [12]:
test_labels = test_labels[:, 1]

In [13]:
test_labels

array([2., 0., 9., ..., 3., 9., 2.])

In [14]:
from sklearn.metrics import accuracy_score
accuracy_score(test_labels, pred)

0.9510357142857143